In [49]:
# Import Dependencies
import requests
import lyricsgenius
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [50]:
# Set Access Tokens and Credentials
# NOTE: Make these imports for the final project!!!!!
genius_token = 'G1xeYkQBe8vtQ6ccWONQiN0Gam2v1Vq4mla9banw9prsfKTI4cWjXq_Jemxxne0w'
spotify_id = 'ddc76410ce5b4eecbbbe641953c92266'
spotify_secret = 'df4777a3cf884456a66d296168097cf3'

In [51]:
# Initialize lyricgenius Class
genius = lyricsgenius.Genius(genius_token)

In [52]:
# Test List of Artists and Songs
test_list = [
    {"song":"Here Comes the Sun", "artist":"The Beetles"},
    {"song":"The Night We Met", "artist":"Lord Huron"},
    {"song":"Peaches", "artist":"The Presidents of the United States of America"},
    {"song":"Longview", "artist":"Green Day"},
    {"song":"Coronor", "artist":"Lando Chill"},
    {"song":"Everlong", "artist":"Foo Fighters"}
]

In [53]:
# Initialize Results List and Error List
results = []
errors = []

# Retrieve Song Lyrics

start = time.time()

for item in test_list:
    try:
        song = genius.search_song(item["song"], artist = item["artist"])
        temp_dict = {"song":item["song"],
                     "artist": item["artist"],
                     "artist_id": "",
                     "album": "",
                     "artist_genre(s)":[], 
                     "album_genre(s)":[],
                     "lyrics": song.lyrics,
                     "audio_features": {}
                    } 
        results.append(temp_dict)
    except:
        temp_dict = {"song":item["song"],
                     "artist": item["artist"],
                     "lyrics": "null"
                    }
        errors.append(temp_dict)
print("finished")
duration = time.time() - start
print(f'Ran in {duration} seconds')
print(f'There were {len(errors)} errors')

Searching for "Here Comes the Sun" by The Beetles...
Done.
Searching for "The Night We Met" by Lord Huron...
Done.
Searching for "Peaches" by The Presidents of the United States of America...
Done.
Searching for "Longview" by Green Day...
Done.
Searching for "Coronor" by Lando Chill...
Done.
Searching for "Everlong" by Foo Fighters...
Done.
finished
Ran in 3.5672590732574463 seconds
There were 0 errors


In [54]:
# View first result item
results[0]

{'song': 'Here Comes the Sun',
 'artist': 'The Beetles',
 'artist_id': '',
 'album': '',
 'artist_genre(s)': [],
 'album_genre(s)': [],
 'lyrics': "[Chorus]\nHere comes the sun, doo da doo doo\nHere comes the sun, and I say\nIt's all right\n\n[Verse 1]\nLittle darling, it's been a long cold lonely winter\nLittle darling, it feels like years since it's been here\n\n[Chorus]\nHere comes the sun, doo da doo doo\nHere comes the sun, and I say\nIt's all right\n\n[Verse 2]\nLittle darling, the smile's returning to the faces\nLittle darling, it seems like years since it's been here\n\n[Chorus]\nHere comes the sun\nHere comes the sun, and I say\nIt's all right\n\n[Bridge]\nSun, sun, sun, here it comes\nSun, sun, sun, here it comes\nSun, sun, sun, here it comes\nSun, sun, sun, here it comes\nSun, sun, sun, here it comes\n[Verse 3]\nLittle darling, I feel that ice is slowly melting\nLittle darling, it seems like years since it's been clear\n\n[Chorus]\nHere comes the sun, doo da doo doo\nHere co

In [55]:
# Initialize Spotipy Class
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=spotify_id, client_secret=spotify_secret))

In [56]:
# Iterate through results add artist Spotify ID
for item in results:
    sp_result = sp.search(item["artist"])
    artist_id = sp_result['tracks']['items'][0]["album"]["artists"][0]["id"]
    item["artist_id"]=artist_id

In [57]:
# View first result item
results[0]

{'song': 'Here Comes the Sun',
 'artist': 'The Beetles',
 'artist_id': '3WrFJ7ztbogyGnTHbHJFl2',
 'album': '',
 'artist_genre(s)': [],
 'album_genre(s)': [],
 'lyrics': "[Chorus]\nHere comes the sun, doo da doo doo\nHere comes the sun, and I say\nIt's all right\n\n[Verse 1]\nLittle darling, it's been a long cold lonely winter\nLittle darling, it feels like years since it's been here\n\n[Chorus]\nHere comes the sun, doo da doo doo\nHere comes the sun, and I say\nIt's all right\n\n[Verse 2]\nLittle darling, the smile's returning to the faces\nLittle darling, it seems like years since it's been here\n\n[Chorus]\nHere comes the sun\nHere comes the sun, and I say\nIt's all right\n\n[Bridge]\nSun, sun, sun, here it comes\nSun, sun, sun, here it comes\nSun, sun, sun, here it comes\nSun, sun, sun, here it comes\nSun, sun, sun, here it comes\n[Verse 3]\nLittle darling, I feel that ice is slowly melting\nLittle darling, it seems like years since it's been clear\n\n[Chorus]\nHere comes the sun, d

In [58]:
# Initialize Spotify API v.2
AUTH_URL = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': spotify_id,
    'client_secret': spotify_secret,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}
BASE_URL = 'https://api.spotify.com/v1/'

In [60]:
# Iterate through results, search by artist ID
# Get genres, add to list
# If results song is in album, get album and audio features

for item in results:
    artist_id = item["artist_id"]
    artist_genres = []
    r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums', 
                 headers=headers, 
                 params={'include_groups': 'album', 'limit': 50})
    d = r.json()
    albums = d["items"]
    
    for x in range(len(albums)):
        album_name = albums[x]["name"]
        
        # Get genres associated with artist - not always consistent w/ album, so iterate though all albums
        artist = sp.artist(albums[x]["artists"][0]["external_urls"]["spotify"])
        genres = artist["genres"]
        print(genres)
        for genre in genres:
            if genre not in artist_genres:
                artist_genres.append(genre)
    print(item["artist"], artist_genres)
    item["artist_genre(s)"] = artist_genres

['beatlesque', 'british invasion', 'classic rock', 'merseybeat', 'psychedelic rock', 'rock']
['beatlesque', 'british invasion', 'classic rock', 'merseybeat', 'psychedelic rock', 'rock']
['beatlesque', 'british invasion', 'classic rock', 'merseybeat', 'psychedelic rock', 'rock']
['beatlesque', 'british invasion', 'classic rock', 'merseybeat', 'psychedelic rock', 'rock']
['beatlesque', 'british invasion', 'classic rock', 'merseybeat', 'psychedelic rock', 'rock']
['beatlesque', 'british invasion', 'classic rock', 'merseybeat', 'psychedelic rock', 'rock']
['beatlesque', 'british invasion', 'classic rock', 'merseybeat', 'psychedelic rock', 'rock']
['beatlesque', 'british invasion', 'classic rock', 'merseybeat', 'psychedelic rock', 'rock']
['beatlesque', 'british invasion', 'classic rock', 'merseybeat', 'psychedelic rock', 'rock']
['beatlesque', 'british invasion', 'classic rock', 'merseybeat', 'psychedelic rock', 'rock']
['beatlesque', 'british invasion', 'classic rock', 'merseybeat', 'psyc

In [43]:
artist

{'external_urls': {'spotify': 'https://open.spotify.com/artist/7jy3rLJdDQY21OgRLCZ9sD'},
 'followers': {'href': None, 'total': 9041044},
 'genres': ['alternative metal',
  'alternative rock',
  'modern rock',
  'permanent wave',
  'post-grunge',
  'rock'],
 'href': 'https://api.spotify.com/v1/artists/7jy3rLJdDQY21OgRLCZ9sD',
 'id': '7jy3rLJdDQY21OgRLCZ9sD',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb9a43b87b50cd3d03544bb3e5',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051749a43b87b50cd3d03544bb3e5',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1789a43b87b50cd3d03544bb3e5',
   'width': 160}],
 'name': 'Foo Fighters',
 'popularity': 81,
 'type': 'artist',
 'uri': 'spotify:artist:7jy3rLJdDQY21OgRLCZ9sD'}

In [61]:
results

[{'song': 'Here Comes the Sun',
  'artist': 'The Beetles',
  'artist_id': '3WrFJ7ztbogyGnTHbHJFl2',
  'album': '',
  'artist_genre(s)': ['beatlesque',
   'british invasion',
   'classic rock',
   'merseybeat',
   'psychedelic rock',
   'rock'],
  'album_genre(s)': [],
  'lyrics': "[Chorus]\nHere comes the sun, doo da doo doo\nHere comes the sun, and I say\nIt's all right\n\n[Verse 1]\nLittle darling, it's been a long cold lonely winter\nLittle darling, it feels like years since it's been here\n\n[Chorus]\nHere comes the sun, doo da doo doo\nHere comes the sun, and I say\nIt's all right\n\n[Verse 2]\nLittle darling, the smile's returning to the faces\nLittle darling, it seems like years since it's been here\n\n[Chorus]\nHere comes the sun\nHere comes the sun, and I say\nIt's all right\n\n[Bridge]\nSun, sun, sun, here it comes\nSun, sun, sun, here it comes\nSun, sun, sun, here it comes\nSun, sun, sun, here it comes\nSun, sun, sun, here it comes\n[Verse 3]\nLittle darling, I feel that ice